In [2]:
%%writefile predictor.py

import os

import numpy as np
import joblib
import pandas as pd

class BankPredictor(object):

    _COLUMN_NAMES = ['age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan', 'contact', 'contacted_month', 'day_of_week', 'duration', 'compaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employees']
    
    def __init__(self, model):
        self._model = model

    def predict(self, instances, **kwargs):
        inputs = pd.DataFrame(data=[instances], columns=self._COLUMN_NAMES)
        outputs = self._model.predict(inputs)
        return outputs.tolist()

    @classmethod
    def from_path(cls, model_dir):
        model_path = os.path.join(model_dir, 'model.joblib')
        model = joblib.load(model_path)
        return cls(model)

Writing predictor.py


In [3]:
from predictor import BankPredictor

model = BankPredictor.from_path('.')
                
instance = [56, "housemaid", "married", "basic.4y", "no", "no", "no", "telephone", "may", "mon", 261, 1, 999, 0, "nonexistent", 1.1, 93.994, -36.4, 4.857, 5191]
print(model.predict(instance))

['no']


In [4]:
%%writefile setup.py

from setuptools import setup
from setuptools import find_packages

REQUIRED_PACKAGES = ['xgboost']

setup(
    name='bank_predict',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    scripts=['predictor.py'])

Writing setup.py


In [5]:
!python setup.py sdist --formats=gztar

running sdist
running egg_info
creating bank_predict.egg-info
writing bank_predict.egg-info/PKG-INFO
writing dependency_links to bank_predict.egg-info/dependency_links.txt
writing requirements to bank_predict.egg-info/requires.txt
writing top-level names to bank_predict.egg-info/top_level.txt
writing manifest file 'bank_predict.egg-info/SOURCES.txt'
reading manifest file 'bank_predict.egg-info/SOURCES.txt'
writing manifest file 'bank_predict.egg-info/SOURCES.txt'

running check


creating bank_predict-0.1
creating bank_predict-0.1/bank_predict.egg-info
copying files to bank_predict-0.1...
copying predictor.py -> bank_predict-0.1
copying setup.py -> bank_predict-0.1
copying bank_predict.egg-info/PKG-INFO -> bank_predict-0.1/bank_predict.egg-info
copying bank_predict.egg-info/SOURCES.txt -> bank_predict-0.1/bank_predict.egg-info
copying bank_predict.egg-info/dependency_links.txt -> bank_predict-0.1/bank_predict.egg-info
copying bank_predict.egg-info/requires.txt -> bank_predict-0.1/bank_

In [6]:
!gsutil cp ./dist/bank_predict-0.1.tar.gz gs://bankapp_gs/bank_predict-0.1.tar.gz

Copying file://./dist/bank_predict-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.1 KiB/  1.1 KiB]                                                
Operation completed over 1 objects/1.1 KiB.                                      


In [7]:
!gcloud beta ai-platform models create BankPredictor --regions us-central1 --enable-console-logging

Using endpoint [https://ml.googleapis.com/]
Created ai platform model [projects/mlbankapp/models/BankPredictor].


In [8]:
!gcloud --quiet beta ai-platform versions create V1 \
  --model BankPredictor \
  --runtime-version 2.7 \
  --python-version 3.7 \
  --origin gs://bankapp_gs/ \
  --package-uris gs://bankapp_gs/bank_predict-0.1.tar.gz \
  --prediction-class predictor.BankPredictor 

Using endpoint [https://us-central1-ml.googleapis.com/]
ERROR: (gcloud.beta.ai-platform.versions.create) FAILED_PRECONDITION: Field: framework Error: Machine type n1-standard-2 does not support CUSTOM_CLASS.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: Machine type n1-standard-2 does not support CUSTOM_CLASS.
    field: framework


In [ ]:
{"instance": [56, "housemaid", "married", "basic.4y", "no", "no", "no", "telephone", "may", "mon", 261, 1, 999, 0, "nonexistent", 1.1, 93.994, -36.4, 4.857, 5191]}

In [8]:
!gcloud ai-platform predict --model BankPredictor --version V1 --json-request input.json

Using endpoint [https://ml.googleapis.com/]
['No']
